In [1]:
import torch
from mambapy.mamba import Mamba, MambaConfig
from torchinfo import summary
from c66 import pp

# B, L, D = 7, 64, 16
B, L, D = 7, 286, 64
pp(B,L,D)

config = MambaConfig(d_model=D, n_layers=1, use_cuda=False)
model = Mamba(config)

B: 7
L: 286
D: 64


In [2]:
model.layers[0].mixer.A_log

Parameter containing:
tensor([[0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        ...,
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726]],
       requires_grad=True)

In [3]:
model.layers[0].mixer.D

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], requires_grad=True)

In [14]:
import torch
from mambapy.mamba import Mamba, MambaConfig
from torchinfo import summary
from c66 import pp

# B, L, D = 7, 64, 16
B, L, D = 7, 286, 64
pp(B,L,D)

config = MambaConfig(d_model=D, n_layers=1, use_cuda=False)
model = Mamba(config)
# .to("cuda")

x = torch.randn(B, L, D)
# .to("cuda")
y = model(x)

assert y.shape == x.shape
pp(x.shape)

B: 7
L: 286
D: 64
------------------------------
In ResidualBlock
x.shape: torch.Size([7, 286, 64])
self.norm(x).shape: torch.Size([7, 286, 64])
------------------------------
------------------------------
In MambaBlock
torch.Size([7, 128, 286])
torch.Size([7, 128, 286])
B, L, ED, N, dt_rank: 7 286 128 16 4
x.shape: torch.Size([7, 286, 128])
delta.shape: torch.Size([7, 286, 128])
A.shape: torch.Size([128, 16])
B.shape: torch.Size([7, 286, 16])
C.shape: torch.Size([7, 286, 16])
z.shape: torch.Size([7, 286, 128])
self.selective_scan
------------------------------
x.shape: torch.Size([7, 286, 64])


In [15]:
print(model)

Mamba(
  (layers): ModuleList(
    (0): ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=64, out_features=256, bias=False)
        (conv1d): Conv1d(128, 128, kernel_size=(4,), stride=(1,), padding=(3,), groups=128)
        (x_proj): Linear(in_features=128, out_features=36, bias=False)
        (dt_proj): Linear(in_features=4, out_features=128, bias=True)
        (out_proj): Linear(in_features=128, out_features=64, bias=False)
      )
      (norm): RMSNorm()
    )
  )
)


In [16]:
print(model.layers[0].mixer)

MambaBlock(
  (in_proj): Linear(in_features=64, out_features=256, bias=False)
  (conv1d): Conv1d(128, 128, kernel_size=(4,), stride=(1,), padding=(3,), groups=128)
  (x_proj): Linear(in_features=128, out_features=36, bias=False)
  (dt_proj): Linear(in_features=4, out_features=128, bias=True)
  (out_proj): Linear(in_features=128, out_features=64, bias=False)
)


In [18]:
model_mamba = model.layers[0].mixer

In [21]:
(B,L,D) = (7,286,64)
with torch.no_grad():
    summary_str = summary(model_mamba, input_size=[(B,L,D)], depth=5, col_names=("input_size", "output_size", "num_params"), verbose=0)
    print(summary_str)

------------------------------
In MambaBlock
torch.Size([7, 128, 286])
torch.Size([7, 128, 286])
B, L, ED, N, dt_rank: 7 286 128 16 4
x.shape: torch.Size([7, 286, 128])
delta.shape: torch.Size([7, 286, 128])
A.shape: torch.Size([128, 16])
B.shape: torch.Size([7, 286, 16])
C.shape: torch.Size([7, 286, 16])
z.shape: torch.Size([7, 286, 128])
self.selective_scan
------------------------------
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MambaBlock                               [7, 286, 64]              [7, 286, 64]              2,816
├─Linear: 1-1                            [7, 286, 64]              [7, 286, 256]             16,384
├─Conv1d: 1-2                            [7, 128, 286]             [7, 128, 289]             640
├─Linear: 1-3                            [7, 286, 128]             [7, 286, 36]              4,608
├─Linear: 1-4                            [7, 286, 128]             [7, 286, 64]              8,192
Total params

In [24]:
input_x = torch.randn(100,286,64)
# input_x = torch.randn(500,286,64)

from calflops import calculate_flops
with torch.no_grad():
        # 使用 calflops 計算 FLOPs，將 args 改為列表
        flops, macs, params = calculate_flops(
            model=model_mamba,
            args=[input_x],  # 使用列表而非元組
            print_results=True  # 顯示逐層結果
        )
        # print(f"Total FLOPs for {fname}: {flops / 1e9:.3f} GFLOPs")
        # print(f"Total Params: {params / 1e6:.3f} M")
        # print(f"Total MACs: {macs / 1e9:.3f} GMACs")
        print(f"Total FLOPs: {flops}")
        print(f"Total Params: {params}")
        print(f"Total MACs: {macs}")

------------------------------
In MambaBlock
torch.Size([100, 128, 286])
torch.Size([100, 128, 286])
B, L, ED, N, dt_rank: 100 286 128 16 4
x.shape: torch.Size([100, 286, 128])
delta.shape: torch.Size([100, 286, 128])
A.shape: torch.Size([128, 16])
B.shape: torch.Size([100, 286, 16])
C.shape: torch.Size([100, 286, 16])
z.shape: torch.Size([100, 286, 128])
self.selective_scan
------------------------------

------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  32.64 K 
fwd MACs:                                    